In [75]:
# loading in modules
import sqlite3
import numpy as np
import pandas as pd

In [25]:
# creating file path
dbfile = 'TRACE.db'
# Create a SQL connection to our SQLite database
conn = sqlite3.connect(dbfile)

# creating cursor
cursor = conn.cursor()

In [ ]:
# reading all table names
table_list = [a for a in cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]

In [29]:
query = """select * from
master_corp_agency 
where cusip_id in 
(select distinct(cusip_id) as uniq_cusip from daily_btds)
;"""
df=(cursor.execute(query)).fetchall()
column_headers = [description[0] for description in cursor.description]
master=pd.DataFrame(df,columns=column_headers)

In [31]:
master.head()

,bond_sym_id,cusip_id,bsym_id,sub_prdct_type,debt_type_cd,issuer_nm,scrty_ds,cpn_rt,cpn_type_cd,trd_rpt_efctv_dt,mtrty_dt,grade,ind_144a,dissem,cnvrb_fl,company_symbol
0,A.GC,00846UAC5,BBG0000KXND3,CORP,S-NT,AGILENT TECHNOLOGIES INC,Senior Unsecured Note,6.5,FXPV,2007-10-29,2017-11-01,I,N,Y,None,A
1,A.GC,00846UAC5,BBG0000KXND3,CORP,S-NT,AGILENT TECHNOLOGIES INC,Senior Unsecured Note,6.5,FXPV,2007-10-29,2017-11-01,I,N,Y,N,A
2,A.GC,00846UAC5,BBG0000KXND3,CORP,SRNT,AGILENT TECHNOLOGIES INC,Senior Note,6.5,FXPV,2007-10-29,2017-11-01,I,N,Y,None,A
3,A.GE,00846UAE1,BBG0000NLVG9,CORP,SRNT,AGILENT TECHNOLOGIES INC,Senior Note,5.5,FXPV,2009-09-14,2015-09-14,I,N,Y,None,A
4,A.GF,00846UAG6,BBG0000HY389,CORP,S-NT,AGILENT TECHNOLOGIES INC,Senior Unsecured Note,5.0,FXPV,2010-07-20,2020-07-15,H,N,Y,N,A


In [4]:
# master=pd.read_csv('master_corp_agency.csv')

In [ ]:
# Just I in grade
# mtrty_dt we get face value plus coupon
# Missing data, interpolate 
# cls price from btds
# join with master
# Return table = monthly log return

In [ ]:
# Build historical yield curve table
# FRED, constant maturity for treasury yield, 3, 10, 30 years

In [5]:
master.head()

,bond_sym_id,cusip_id,bsym_id,sub_prdct_type,debt_type_cd,issuer_nm,scrty_ds,cpn_rt,cpn_type_cd,trd_rpt_efctv_dt,mtrty_dt,grade,ind_144a,dissem,cnvrb_fl,company_symbol
0,3264R4938696,88577CAA9,BBG00RF7NLB4,CORP,SC-CAT,3264 RE LTD,Secured Insurance Linked Security,9.75,FRPV,2020-01-17,2023-02-07,H,Y,Y,N,3264R4
1,A.GC,00846UAC5,BBG0000KXND3,CORP,S-NT,AGILENT TECHNOLOGIES INC,Senior Unsecured Note,6.50,FXPV,2007-10-29,2017-11-01,I,N,Y,NaN,A
2,A.GC,00846UAC5,BBG0000KXND3,CORP,S-NT,AGILENT TECHNOLOGIES INC,Senior Unsecured Note,6.50,FXPV,2007-10-29,2017-11-01,I,N,Y,N,A
3,A.GC,00846UAC5,BBG0000KXND3,CORP,SRNT,AGILENT TECHNOLOGIES INC,Senior Note,6.50,FXPV,2007-10-29,2017-11-01,I,N,Y,NaN,A
4,A.GD,00846UAD3,BBG0000NNY17,CORP,SRNT,Agilent Tech,Senior Note,4.45,FXPV,2009-09-14,2012-09-14,I,N,Y,NaN,A


In [42]:
# Filtering for Investment Grade Bonds
master_inv=master.loc[master['grade']=='I']
master_inv.shape

(120010, 16)

In [33]:
df_btds=pd.read_csv('daily_btds_144A.csv')

In [34]:
df_btds.head()

,trans_dt,bond_sym_id,cusip_id,bsym,sub_prd_type,wis_fl,high_pr,high_yld_sign_cd,high_yld,low_pr,low_yld_sign_cd,low_yld,close_pr,close_yld_sign_cd,close_yld,company_symbol
0,2015-01-22,NaN,04318AAA4,NaN,CORP,NaN,101.50,NaN,0.0,100.375,NaN,0.0,101.500,NaN,0.0,NaN
1,2015-02-06,NaN,30706VAA3,NaN,CORP,NaN,103.50,NaN,0.0,100.500,NaN,0.0,103.000,NaN,0.0,NaN
2,2015-02-06,NaN,30706VAC9,NaN,CORP,NaN,103.25,NaN,0.0,100.500,NaN,0.0,102.500,NaN,0.0,NaN
3,2015-02-10,NaN,19240CAA1,NaN,CORP,NaN,101.00,NaN,0.0,100.125,NaN,0.0,100.125,NaN,0.0,NaN
4,2015-02-11,NaN,00214TAA6,NaN,CORP,NaN,102.50,NaN,0.0,101.000,NaN,0.0,102.000,NaN,0.0,NaN


In [68]:
master_inv.shape

(120010, 16)

In [69]:
df_btds.shape

(3198642, 16)

In [35]:
btds_cusips=pd.DataFrame(df_btds['cusip_id'].unique())

In [36]:
master_cusips=pd.DataFrame(master_inv['cusip_id'].unique())

In [37]:
len(master_inv['cusip_id'].unique())

139411

In [43]:
merged_df=df_btds.merge(master_inv,on='cusip_id')

In [44]:
merged_df.columns

Index(['trans_dt', 'bond_sym_id_x', 'cusip_id', 'bsym', 'sub_prd_type',
       'wis_fl', 'high_pr', 'high_yld_sign_cd', 'high_yld', 'low_pr',
       'low_yld_sign_cd', 'low_yld', 'close_pr', 'close_yld_sign_cd',
       'close_yld', 'company_symbol_x', 'bond_sym_id_y', 'bsym_id',
       'sub_prdct_type', 'debt_type_cd', 'issuer_nm', 'scrty_ds', 'cpn_rt',
       'cpn_type_cd', 'trd_rpt_efctv_dt', 'mtrty_dt', 'grade', 'ind_144a',
       'dissem', 'cnvrb_fl', 'company_symbol_y'],
      dtype='object')

In [45]:
merged_df.shape

(81501, 31)

In [54]:
merged_df['mtrty_dt'].sort_values(ascending=False)

17611    3999-12-31
20273    3999-12-31
20275    3999-12-31
20276    3999-12-31
20277    3999-12-31
            ...    
5381     2015-03-19
5382     2015-03-19
5383     2015-03-19
28331    2015-01-12
55815    2013-10-29
Name: mtrty_dt, Length: 81501, dtype: object

In [63]:
merged_df_active=merged_df.loc[merged_df['mtrty_dt']>'2024-02-07']

In [64]:
merged_df_active.shape

(34283, 31)

In [65]:
merged_df_active.head()

,trans_dt,bond_sym_id_x,cusip_id,bsym,sub_prd_type,wis_fl,high_pr,high_yld_sign_cd,high_yld,low_pr,...,scrty_ds,cpn_rt,cpn_type_cd,trd_rpt_efctv_dt,mtrty_dt,grade,ind_144a,dissem,cnvrb_fl,company_symbol_y
2,2021-04-08,ADVO4664868,007589AA2,BBG00LNJJ9W1,CORP,NaN,112.986,NaN,1.865527,112.886,...,Senior Unsecured Bond,3.829,FXPV,2021-04-01,2028-08-15,I,Y,Y,N,ADVO
3,2021-04-08,ADVO4664868,007589AA2,BBG00LNJJ9W1,CORP,NaN,112.986,NaN,1.865527,112.886,...,Unsecured Bond,3.829,FXPV,2018-08-07,2028-08-15,I,N,Y,N,ADVO
4,2021-04-14,ADVO4664868,007589AA2,BBG00LNJJ9W1,CORP,NaN,111.991,NaN,2.004030,111.991,...,Senior Unsecured Bond,3.829,FXPV,2021-04-01,2028-08-15,I,Y,Y,N,ADVO
5,2021-04-14,ADVO4664868,007589AA2,BBG00LNJJ9W1,CORP,NaN,111.991,NaN,2.004030,111.991,...,Unsecured Bond,3.829,FXPV,2018-08-07,2028-08-15,I,N,Y,N,ADVO
6,2021-04-19,ADVO4664868,007589AA2,BBG00LNJJ9W1,CORP,NaN,111.968,NaN,2.004176,111.933,...,Senior Unsecured Bond,3.829,FXPV,2021-04-01,2028-08-15,I,Y,Y,N,ADVO


In [76]:
bond_prices=merged_df_active[['trans_dt','cusip_id','close_pr']]

In [77]:
bond_prices

,trans_dt,cusip_id,close_pr
2,2021-04-08,007589AA2,112.986
3,2021-04-08,007589AA2,112.986
4,2021-04-14,007589AA2,111.991
5,2021-04-14,007589AA2,111.991
6,2021-04-19,007589AA2,111.968
...,...,...,...
81496,2023-09-06,485260BM4,76.069
81497,2023-09-06,485260BM4,76.069
81498,2023-09-13,485260BM4,76.841
81499,2023-09-13,485260BM4,76.841


In [78]:
# Convert trans_dt to datetime
bond_prices['trans_dt'] = pd.to_datetime(bond_prices['trans_dt'])

# Ensure data is sorted
bond_prices = bond_prices.sort_values(by=['cusip_id', 'trans_dt'])

# Remove duplicates by taking the mean or last (here we use mean for example)
bond_prices = bond_prices.groupby(['trans_dt', 'cusip_id']).mean().reset_index()

# Set the index to trans_dt
bond_prices.set_index('trans_dt', inplace=True)

# Find the min and max dates to create a full date range
min_date = bond_prices.index.min()
max_date = bond_prices.index.max()
date_range = pd.date_range(min_date, max_date, freq='D')

# Reindex the dataframe for each cusip_id to include all days in the range
# and forward fill the missing data
bond_prices = (
    bond_prices
    .groupby('cusip_id')
    .apply(lambda x: x.reindex(date_range).ffill().reset_index())
    .reset_index(drop=True)
)

# Rename 'index' back to 'trans_dt'
bond_prices.rename(columns={'index': 'trans_dt'}, inplace=True)

# Calculate log returns
bond_prices['log_returns'] = np.log(bond_prices['close_pr'] / bond_prices['close_pr'].shift(1))

# Average log returns by cusip_id
average_log_returns = bond_prices.groupby('cusip_id')['log_returns'].mean().reset_index()


/var/folders/lt/smy1t7_91rl3z_bw_803wb0h0000gn/T/ipykernel_96951/1908335086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bond_prices['trans_dt'] = pd.to_datetime(bond_prices['trans_dt'])


In [79]:
average_log_returns

,cusip_id,log_returns
0,00138GAB5,-0.000032
1,00138GAC3,0.000027
2,007589AA2,-0.000208
3,02109TAD4,-0.001397
4,025816CN7,-0.000128
...,...,...
131,U03160AX3,-0.000102
132,U0926LAA9,0.000007
133,U0942JAA4,0.000000
134,U1109MAR7,0.000000


In [13]:
len(df_btds['cusip_id'].unique())

17170

In [27]:
type(df_btds.at[0,'trans_dt'])

str

In [29]:
type(df_btds.at[0,'trans_dt'])

pandas._libs.tslibs.timestamps.Timestamp

In [28]:
df_btds['trans_dt']=pd.to_datetime(df_btds['trans_dt'])

In [33]:
df_btds['month_year']=df_btds['trans_dt'].dt.month.astype(str)+"_"+df_btds['trans_dt'].dt.year.astype(str)

In [34]:
df_btds['month_year']

0          1_2015
1          2_2015
2          2_2015
3          2_2015
4          2_2015
            ...  
3198637    2_2019
3198638    2_2019
3198639    5_2023
3198640    6_2023
3198641    8_2023
Name: month_year, Length: 3198642, dtype: object

In [37]:
df_btds.groupby(['month_year'])['cusip_id'].count()

month_year
10_2015    19608
10_2016    19863
10_2017    23866
10_2018    29312
10_2019    30697
           ...  
9_2019     27954
9_2020     35976
9_2021     43369
9_2022     45238
9_2023     39357
Name: cusip_id, Length: 106, dtype: int64

In [4]:
bond_factors=pd.read_csv('./Data/bond_factors.csv')

In [5]:
bond_factors.head()

,date,Value,Market Value*,Amount Outstanding*,Duration*,Bond Age,Book-to-price,Credit Spread,Yield to maturity,VaR(5%),...,Term-beta,VaR(10%),Skewness*,3m. Momentum,6m. Momentum,9m. Momentum,12m. Momentum,Equity Momentum,Short Term Reversal,Long Term Reversal
0,1985-02-28,-0.020768,0.000726,0.000701,0.029944,-0.000637,-0.011800,0.022666,0.017791,-0.015418,...,-0.026047,-0.019713,0.000832,NaN,-0.028908,-0.026502,-0.024953,0.001375,NaN,-0.005982
1,1985-03-31,-0.001099,0.007648,0.004340,-0.001567,-0.002664,0.005763,0.015137,0.012812,0.004064,...,0.016010,0.014194,0.005158,NaN,NaN,NaN,NaN,0.006118,0.015755,0.007368
2,1985-04-30,0.001897,-0.004064,-0.006550,-0.003292,0.001213,0.007721,0.003344,0.006584,0.012915,...,0.011307,0.017479,-0.001383,NaN,NaN,NaN,NaN,0.005285,0.003172,0.002893
3,1985-05-31,0.026860,0.003696,-0.008432,-0.033733,-0.000387,0.023527,-0.023602,-0.007869,0.021000,...,0.048765,0.032031,-0.009372,NaN,NaN,NaN,NaN,-0.004484,-0.023631,0.019986
4,1985-06-30,-0.002478,0.013160,0.005771,0.006049,0.006970,0.014167,0.013160,0.010946,0.010664,...,-0.007487,0.005905,0.000753,0.002415,NaN,NaN,NaN,-0.012729,0.014910,-0.002410


In [6]:
bond_firm_factors=pd.read_csv('./Data/bond_firm_factors.csv')

In [11]:
bond_firm_factors.columns

Index(['date', 'Value', 'Market Value*', 'Amount Outstanding*', 'Duration*',
       'Bond Age', 'Book-to-price', 'Credit Spread', 'Yield to maturity',
       'VaR(5%)', 'Volatility', 'ES(10%)', 'Default-beta', 'Term-beta',
       'VaR(10%)', 'Skewness*', '3m. Momentum', '6m. Momentum', '9m. Momentum',
       '12m. Momentum', 'Equity Momentum', 'Short Term Reversal',
       'Long Term Reversal'],
      dtype='object')

In [7]:
bond_firm_factors.head()

,date,Value,Market Value*,Amount Outstanding*,Duration*,Bond Age,Book-to-price,Credit Spread,Yield to maturity,VaR(5%),...,Term-beta,VaR(10%),Skewness*,3m. Momentum,6m. Momentum,9m. Momentum,12m. Momentum,Equity Momentum,Short Term Reversal,Long Term Reversal
0,1985-02-28,-0.008193,0.010543,0.010151,0.017169,0.003792,-0.010576,0.004922,0.000062,-0.017357,...,-0.016051,-0.014351,0.000490,NaN,-0.011871,-0.011141,-0.008864,0.002658,NaN,-0.003509
1,1985-03-31,-0.001512,-0.003335,-0.003432,-0.002773,-0.001762,0.002099,0.003432,0.003400,0.004021,...,0.003090,0.002225,0.003159,NaN,NaN,NaN,NaN,-0.000638,0.005045,0.003433
2,1985-04-30,0.000291,0.001172,0.000454,-0.000171,-0.000925,0.003448,0.000760,0.005383,-0.004547,...,0.000825,0.000618,-0.008631,NaN,NaN,NaN,NaN,0.005823,0.006169,-0.005118
3,1985-05-31,0.006711,-0.007488,-0.009067,-0.024065,0.003893,0.020272,0.005627,0.009315,0.030455,...,0.025126,0.021058,0.002082,NaN,NaN,NaN,NaN,-0.011952,-0.005399,0.004307
4,1985-06-30,-0.004259,0.003979,0.003116,0.006696,0.000974,-0.001852,0.003520,0.001228,0.000862,...,-0.002661,-0.004051,0.000469,0.002126,NaN,NaN,NaN,-0.002441,0.005181,0.001013


In [10]:
bond_firm_factors.shape

(443, 23)